# I fucking hate `pandas` and `sklearn` for their messy and inconsistent interfaces
# for their opposition to any extensions of their libraries
# and all that crap.
# I can't even sleep because of how much I hate them.
# I was trying to write things according to their API
# implementing BaseEstimator and TransformerMixin
# and using pipes
# but that makes things even worser
# because of uglyness of this environment!

# I wanted to try pretty obvious tricks and models
# but this environment makes it impossible

In [1]:
import pandas as pd
import numpy as np
import dateutil.parser
import datetime
import sklearn.preprocessing
import collections
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, SelectPercentile,chi2, f_classif, SelectFpr
from sklearn.pipeline import FeatureUnion

%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [4]:
def date_to_ymd(x):
    res = (np.nan, np.nan, np.nan)
    if isinstance(x, str):
        try:
            x = dateutil.parser.parse(x)
        except:
            pass
    if isinstance(x, datetime.date):
            res = (x.year, x.month, x.day)
    return res

In [5]:
X_train.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)
X_test.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)

In [6]:
(X_train.loc[:,'date_recorded_year'],
 X_train.loc[:,'date_recorded_month'],
 X_train.loc[:,'date_recorded_day']) = zip(*X_train.loc[:,'date_recorded'].map(date_to_ymd))
(X_test.loc[:,'date_recorded_year'],
 X_test.loc[:,'date_recorded_month'],
 X_test.loc[:,'date_recorded_day']) = zip(*X_test.loc[:,'date_recorded'].map(date_to_ymd))

X_train.drop('date_recorded', axis=1, inplace=True)
X_test.drop('date_recorded', axis=1, inplace=True)

In [7]:
X_train.loc[:,'age'] = X_train.loc[:,'date_recorded_year'] - X_train.loc[:,'construction_year']
X_test.loc[:,'age'] = X_test.loc[:,'date_recorded_year'] - X_test.loc[:,'construction_year']

In [8]:
X_train.loc[:,'amount_per_man'] = (X_train.loc[:,'amount_tsh']/X_train.loc[:,'population']).fillna(0)
X_test.loc[:,'amount_per_man'] = (X_test.loc[:,'amount_tsh']/X_test.loc[:,'population']).fillna(0)

In [9]:
X_train.loc[X_train.loc[:,'permit'] == True, 'permit'] = 1
X_train.loc[X_train.loc[:,'permit'] == False, 'permit'] = 0
X_train.loc[:,'permit'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'permit'] == True, 'permit'] = 1
X_test.loc[X_test.loc[:,'permit'] == False, 'permit'] = 0
X_test.loc[:,'permit'].fillna(0.5, inplace=True)

In [10]:
X_train.loc[X_train.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_train.loc[X_train.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_train.loc[:,'public_meeting'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_test.loc[X_test.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_test.loc[:,'public_meeting'].fillna(0.5, inplace=True)

In [11]:
X_train.loc[:,'wpt_name'] = X_train.loc[:,'wpt_name'].notnull() * 1
X_test.loc[:,'wpt_name'] = X_test.loc[:,'wpt_name'].notnull() * 1

In [12]:
X_train.loc[:,'source'] = (
    X_train.loc[:,'source'] + ' '
    + X_train.loc[:,'source_type'] + ' '
    + X_train.loc[:,'source_class']
)
X_test.loc[:,'source'] = (
    X_test.loc[:,'source'] + ' '
    + X_test.loc[:,'source_type'] + ' '
    + X_test.loc[:,'source_class']
)

In [13]:
X_train.loc[:,'quality'] = (
    X_train.loc[:,'water_quality'] + ' '
    + X_train.loc[:,'quality_group']
)
X_test.loc[:,'quality'] = (
    X_test.loc[:,'water_quality'] + ' '
    + X_test.loc[:,'quality_group']
)

In [14]:
X_train.loc[:,'waterpoint_type'] = (
    X_train.loc[:,'waterpoint_type'] + ' '
    + X_train.loc[:,'waterpoint_type_group']
)
X_test.loc[:,'waterpoint_type'] = (
    X_test.loc[:,'waterpoint_type'] + ' '
    + X_test.loc[:,'waterpoint_type_group']
)

In [15]:
X_train.loc[:,'quantity'] = (
    X_train.loc[:,'quantity'] + ' '
    + X_train.loc[:,'quantity_group']
)
X_test.loc[:,'quantity'] = (
    X_test.loc[:,'quantity'] + ' '
    + X_test.loc[:,'quantity_group']
)

In [16]:
X_train.loc[:,'management'] = (
    X_train.loc[:,'management'] + ' '
    + X_train.loc[:,'management_group']
)
X_test.loc[:,'management'] = (
    X_test.loc[:,'management'] + ' '
    + X_test.loc[:,'management_group']
)

In [17]:
X_train.loc[:,'payment'] = (
    X_train.loc[:,'payment'] + ' '
    + X_train.loc[:,'payment_type']
)
X_test.loc[:,'payment'] = (
    X_test.loc[:,'payment'] + ' '
    + X_test.loc[:,'payment_type']
)

In [18]:
X_train.loc[:,'extraction_type'] = (
    X_train.loc[:,'extraction_type'] + ' '
    + X_train.loc[:,'extraction_type_group'] + ' '
    + X_train.loc[:,'extraction_type_class']
)
X_test.loc[:,'extraction_type'] = (
    X_test.loc[:,'extraction_type'] + ' '
    + X_test.loc[:,'extraction_type_group'] + ' '
    + X_test.loc[:,'extraction_type_class']
)

In [19]:
drop_cats = ['source_type', 'source_class',
              'water_quality', 'quality_group',
              'quantity_group', 'waterpoint_type_group',
              'payment_type', 'extraction_type_group',
              'extraction_type_class', 'management_group'
              ]
X_train.drop(drop_cats, axis=1, inplace=True, errors='ignore')
X_test.drop(drop_cats, axis=1, inplace=True, errors='ignore')

In [20]:
X_train.iloc[0,:]

amount_tsh                                              6000
funder                                                 Roman
gps_height                                              1390
installer                                              Roman
longitude                                            34.9381
latitude                                            -9.85632
wpt_name                                                   1
basin                                             Lake Nyasa
subvillage                                          Mnyusi B
region                                                Iringa
region_code                                               11
district_code                                              5
lga                                                   Ludewa
ward                                                Mundindi
population                                               109
public_meeting                                             1
scheme_management       

In [21]:
text_features = ['funder', 'installer', 'basin',
                'subvillage', 'region', 'region_code', 'district_code',
                'lga', 'ward', 
                 'quality',
                 #'water_quality', 'quality_group',
                'scheme_management', 'scheme_name',
                'extraction_type', #'extraction_type_group',
                'management',# 'management_group', 
                 'payment',# 'payment_type',
                'quantity',# 'quantity_group',
                'source',# 'source_type', 'source_class',
                 'waterpoint_type', #'waterpoint_type_group',
                #'date_recorded_year', 'construction_year',
               ]
text_features_vectorizers = dict()
text_feats_train = []
text_feats_test = []

In [22]:
y_enc1 = sklearn.preprocessing.LabelEncoder().fit(y_train.iloc[:,1])
ytr = y_enc1.transform(y_train.iloc[:,1])

In [23]:
for cat in text_features:
    if X_train.loc[:,cat].dtype == object:
        X_train.loc[:,cat].fillna('unknown', inplace=True)
        X_train.loc[X_train.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
        X_test.loc[:,cat].fillna('unknown', inplace=True)
        X_test.loc[X_test.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
    else:
        X_train.loc[:,cat].fillna(0, inplace=True)
        X_test.loc[:,cat].fillna(0, inplace=True)
        X_train.loc[:cat] = X_train.loc[:cat].astype(str)
        X_test.loc[:cat] = X_test.loc[:cat].astype(str)
    v = TfidfVectorizer(ngram_range=(1,2))
    v.fit(X_test.loc[:,cat])
    tr = v.transform(X_train.loc[:,cat])
    te = v.transform(X_test.loc[:,cat])
    #kbest = SelectKBest(chi2, min(8, tr.shape[1])).fit(tr, ytr)
    sel = FeatureUnion([
        ('anova', SelectFpr(f_classif, 1e-19)), # yeah, i'm gonna be very pretentious!
        ('chi2', SelectKBest(chi2, 3))
    ])
    sel.fit(tr, ytr)
    tr = sel.transform(tr)
    te = sel.transform(te)
    tr = pd.DataFrame(tr.toarray()).add_prefix(cat + '_')
    te = pd.DataFrame(te.toarray()).add_prefix(cat + '_')
    text_feats_train.append(tr)
    text_feats_test.append(te)

# TODO: Now select from a whole bunch of these

X_train = pd.concat([X_train.drop(text_features, axis=1, errors='ignore')] + text_feats_train,
                    axis=1)
X_test = pd.concat([X_test.drop(text_features, axis=1, errors='ignore')] + text_feats_test,
                    axis=1)


/home/god/py/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] are constant.
  UserWarning)
/home/god/py/lib/python3.6/site-pac

In [24]:
X_train.head()

,amount_tsh,gps_height,longitude,latitude,wpt_name,population,public_meeting,permit,construction_year,date_recorded_year,...,waterpoint_type_9,waterpoint_type_10,waterpoint_type_11,waterpoint_type_12,waterpoint_type_13,waterpoint_type_14,waterpoint_type_15,waterpoint_type_16,waterpoint_type_17,waterpoint_type_18
0,6000.0,1390,34.93809275,-9.85632177,1,109,1.0,0.0,1999,2011,...,0.0,0.0,0.0,0.549138,0.308775,0.000000,0.0,0.000000,0.000000,0.0
1,0.0,1399,34.6987661,-2.14746569,1,280,0.5,1.0,2010,2013,...,0.0,0.0,0.0,0.549138,0.308775,0.000000,0.0,0.000000,0.000000,0.0
2,25.0,686,37.46066446,-3.82132853,1,250,1.0,1.0,2009,2013,...,0.0,0.0,0.0,0.395413,0.000000,0.420692,0.0,0.420692,0.420692,0.0
3,0.0,263,38.48616088,-11.15529772,1,58,1.0,1.0,1986,2013,...,0.0,0.0,0.0,0.395413,0.000000,0.420692,0.0,0.420692,0.420692,0.0
4,0.0,0,31.13084671,-1.82535885,1,0,1.0,1.0,0,2011,...,0.0,0.0,0.0,0.549138,0.308775,0.000000,0.0,0.000000,0.000000,0.0


In [25]:
[(c, X_train[c].dtype) for c in X_train.columns if not np.issubdtype(X_train[c].dtype, np.number)]

[('amount_tsh', dtype('O')),
 ('gps_height', dtype('O')),
 ('longitude', dtype('O')),
 ('latitude', dtype('O')),
 ('wpt_name', dtype('O')),
 ('population', dtype('O')),
 ('public_meeting', dtype('O')),
 ('permit', dtype('O')),
 ('construction_year', dtype('O')),
 ('date_recorded_year', dtype('O')),
 ('date_recorded_month', dtype('O')),
 ('date_recorded_day', dtype('O')),
 ('age', dtype('O')),
 ('amount_per_man', dtype('O'))]

In [26]:
def split_train_cv_test(X, y, proportions=(.75, .25/2, .25/2)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        # print(cidx)
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    parts = xparts + yparts
    return parts


xtr = X_train # ref
xte = X_test # ref
xtr, xcv, xva, ytr, ycv, yva = (
    split_train_cv_test(xtr, ytr, proportions=(.7, .15, .15))
)
#y_bin = sklearn.preprocessing.LabelBinarizer().fit(ytr)
#display(
#    np.where(ytr != 0)[0][:40],
#    np.where(ycv != 0)[0][:40],
#    np.where(yva != 0)[0][:40],)
#ytr = y_bin.transform(ytr)
#ycv = y_bin.transform(ycv)
#yva = y_bin.transform(yva)

In [27]:
def y_to_df(y, ids):
    #y = y_bin.inverse_transform(y)
    y = y_enc1.inverse_transform(y)
    df = pd.DataFrame({'status_group': y}, index=ids)
    df.index.name = 'id'
    return df

In [28]:
xtr.columns.tolist()

['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'wpt_name',
 'population',
 'public_meeting',
 'permit',
 'construction_year',
 'date_recorded_year',
 'date_recorded_month',
 'date_recorded_day',
 'age',
 'amount_per_man',
 'funder_0',
 'funder_1',
 'funder_2',
 'funder_3',
 'funder_4',
 'funder_5',
 'funder_6',
 'funder_7',
 'funder_8',
 'funder_9',
 'funder_10',
 'funder_11',
 'funder_12',
 'funder_13',
 'funder_14',
 'funder_15',
 'funder_16',
 'funder_17',
 'funder_18',
 'funder_19',
 'funder_20',
 'funder_21',
 'funder_22',
 'funder_23',
 'funder_24',
 'funder_25',
 'funder_26',
 'funder_27',
 'funder_28',
 'funder_29',
 'funder_30',
 'funder_31',
 'funder_32',
 'funder_33',
 'funder_34',
 'funder_35',
 'funder_36',
 'funder_37',
 'funder_38',
 'funder_39',
 'funder_40',
 'funder_41',
 'funder_42',
 'funder_43',
 'funder_44',
 'funder_45',
 'funder_46',
 'funder_47',
 'funder_48',
 'funder_49',
 'funder_50',
 'funder_51',
 'funder_52',
 'funder_53',
 'funder_54',
 'funde

In [29]:
X_train.to_csv('X_train_prepr2.csv')
X_test.to_csv('X_test_prepr2.csv')
# pd.DataFrame(ytr, columns=y_enc1.classes_).to_csv('y_train_prepr2.csv')
pd.Series(ytr).to_csv('y_train_prepr2.csv')
pd.Series(y_enc1.classes_).to_csv('y_classes2.csv')

In [30]:
import catboost

In [ ]:
clf = catboost.CatBoostClassifier(1800, loss_function='MultiClass',
                                  eval_metric='Accuracy',
                                  calc_feature_importances=True)

In [ ]:
clf.fit(xtr, ytr, cat_features=[xtr.columns.get_loc(c) for c in text_features if c in xtr.columns],
        eval_set=(xcv, ycv), verbose=True)

Borders generated
0:	learn 0.7030160181	test 0.7016165245	bestTest 0.7016165245	passed: 0.562 sec	total: 1.19s	remaining: 35m 41s
1:	learn 0.7042907307	test 0.7021778177	bestTest 0.7021778177	passed: 0.515 sec	total: 1.71s	remaining: 25m 32s
2:	learn 0.7031603252	test 0.7018410418	bestTest 0.7021778177	passed: 0.526 sec	total: 2.23s	remaining: 22m 16s
3:	learn 0.7029919669	test 0.7015042658	bestTest 0.7021778177	passed: 0.527 sec	total: 2.76s	remaining: 20m 38s
4:	learn 0.7029198134	test 0.7015042658	bestTest 0.7021778177	passed: 0.531 sec	total: 3.29s	remaining: 19m 41s
5:	learn 0.7036413488	test 0.7028513696	bestTest 0.7028513696	passed: 0.547 sec	total: 3.84s	remaining: 19m 7s
6:	learn 0.7032565299	test 0.7026268523	bestTest 0.7028513696	passed: 0.503 sec	total: 4.34s	remaining: 18m 31s
7:	learn 0.7030641204	test 0.702402335	bestTest 0.7028513696	passed: 0.494 sec	total: 4.83s	remaining: 18m 3s
8:	learn 0.704338833	test 0.7037494387	bestTest 0.7037494387	passed: 0.528 sec	total: 5.3

73:	learn 0.7212468132	test 0.7210372699	bestTest 0.7210372699	passed: 0.431 sec	total: 37.2s	remaining: 14m 28s
74:	learn 0.7214632738	test 0.7213740458	bestTest 0.7213740458	passed: 0.537 sec	total: 37.8s	remaining: 14m 28s
75:	learn 0.7224012699	test 0.7223843736	bestTest 0.7223843736	passed: 0.47 sec	total: 38.2s	remaining: 14m 27s
76:	learn 0.7240367502	test 0.7241805119	bestTest 0.7241805119	passed: 0.579 sec	total: 38.8s	remaining: 14m 28s
77:	learn 0.724277262	test 0.7245172878	bestTest 0.7245172878	passed: 0.426 sec	total: 39.2s	remaining: 14m 26s
78:	learn 0.7256481793	test 0.7253030983	bestTest 0.7253030983	passed: 0.469 sec	total: 39.7s	remaining: 14m 24s
79:	learn 0.7260570494	test 0.725415357	bestTest 0.725415357	passed: 0.571 sec	total: 40.3s	remaining: 14m 25s
80:	learn 0.7258165376	test 0.7246295465	bestTest 0.725415357	passed: 0.432 sec	total: 40.7s	remaining: 14m 23s
81:	learn 0.7265140218	test 0.7255276156	bestTest 0.7255276156	passed: 0.504 sec	total: 41.2s	remaini

146:	learn 0.7412092934	test 0.7436012573	bestTest 0.7436012573	passed: 0.469 sec	total: 1m 16s	remaining: 14m 19s
147:	learn 0.7412573957	test 0.7434889987	bestTest 0.7436012573	passed: 0.584 sec	total: 1m 17s	remaining: 14m 19s
148:	learn 0.741425754	test 0.7437135159	bestTest 0.7437135159	passed: 0.523 sec	total: 1m 17s	remaining: 14m 19s
149:	learn 0.7415219587	test 0.7441625505	bestTest 0.7441625505	passed: 0.66 sec	total: 1m 18s	remaining: 14m 20s
150:	learn 0.74195488	test 0.7443870678	bestTest 0.7443870678	passed: 0.742 sec	total: 1m 18s	remaining: 14m 22s
151:	learn 0.7423637501	test 0.7453973956	bestTest 0.7453973956	passed: 0.671 sec	total: 1m 19s	remaining: 14m 23s
152:	learn 0.7425321083	test 0.7453973956	bestTest 0.7453973956	passed: 0.67 sec	total: 1m 20s	remaining: 14m 24s
153:	learn 0.742484006	test 0.744948361	bestTest 0.7453973956	passed: 0.663 sec	total: 1m 20s	remaining: 14m 25s
154:	learn 0.7425080571	test 0.7456219129	bestTest 0.7456219129	passed: 0.77 sec	total:

218:	learn 0.7490259272	test 0.7483161203	bestTest 0.7485406376	passed: 0.565 sec	total: 1m 58s	remaining: 14m 16s
219:	learn 0.7492904902	test 0.7479793444	bestTest 0.7485406376	passed: 0.532 sec	total: 1m 59s	remaining: 14m 16s
220:	learn 0.7490980807	test 0.7482038617	bestTest 0.7485406376	passed: 0.571 sec	total: 1m 59s	remaining: 14m 15s
221:	learn 0.749266439	test 0.7482038617	bestTest 0.7485406376	passed: 0.617 sec	total: 2m	remaining: 14m 15s
222:	learn 0.7490980807	test 0.7482038617	bestTest 0.7485406376	passed: 0.543 sec	total: 2m	remaining: 14m 15s
223:	learn 0.7494347972	test 0.7480916031	bestTest 0.7485406376	passed: 0.595 sec	total: 2m 1s	remaining: 14m 15s
224:	learn 0.7493145413	test 0.7474180512	bestTest 0.7485406376	passed: 0.546 sec	total: 2m 2s	remaining: 14m 14s
225:	learn 0.7493866949	test 0.7479793444	bestTest 0.7485406376	passed: 0.593 sec	total: 2m 2s	remaining: 14m 14s
226:	learn 0.7496272067	test 0.7478670858	bestTest 0.7485406376	passed: 0.62 sec	total: 2m 3

291:	learn 0.7549665689	test 0.7535922766	bestTest 0.7540413112	passed: 0.477 sec	total: 2m 40s	remaining: 13m 48s
292:	learn 0.7553273366	test 0.7535922766	bestTest 0.7540413112	passed: 0.492 sec	total: 2m 40s	remaining: 13m 47s
293:	learn 0.7553273366	test 0.7539290525	bestTest 0.7540413112	passed: 0.552 sec	total: 2m 41s	remaining: 13m 47s
294:	learn 0.755519746	test 0.7542658285	bestTest 0.7542658285	passed: 0.541 sec	total: 2m 41s	remaining: 13m 46s
295:	learn 0.7559045649	test 0.7546026044	bestTest 0.7546026044	passed: 0.572 sec	total: 2m 42s	remaining: 13m 45s
296:	learn 0.755784309	test 0.7543780871	bestTest 0.7546026044	passed: 0.542 sec	total: 2m 43s	remaining: 13m 45s
297:	learn 0.7559526673	test 0.7541535698	bestTest 0.7546026044	passed: 0.511 sec	total: 2m 43s	remaining: 13m 44s
298:	learn 0.7561691279	test 0.7541535698	bestTest 0.7546026044	passed: 0.665 sec	total: 2m 44s	remaining: 13m 44s
299:	learn 0.7562653326	test 0.7541535698	bestTest 0.7546026044	passed: 0.673 sec	

...

651:	learn 0.7732214152	test 0.7702065559	bestTest 0.7704310732	passed: 0.582 sec	total: 6m 19s	remaining: 11m 7s
652:	learn 0.7733416711	test 0.7704310732	bestTest 0.7704310732	passed: 0.578 sec	total: 6m 19s	remaining: 11m 6s
653:	learn 0.7732935687	test 0.7702065559	bestTest 0.7704310732	passed: 0.547 sec	total: 6m 20s	remaining: 11m 6s
654:	learn 0.7734138246	test 0.7703188145	bestTest 0.7704310732	passed: 0.591 sec	total: 6m 20s	remaining: 11m 5s
655:	learn 0.7736062341	test 0.7705433318	bestTest 0.7705433318	passed: 0.53 sec	total: 6m 21s	remaining: 11m 4s
656:	learn 0.7736783876	test 0.7705433318	bestTest 0.7705433318	passed: 0.547 sec	total: 6m 21s	remaining: 11m 4s
657:	learn 0.7737024388	test 0.7705433318	bestTest 0.7705433318	passed: 0.526 sec	total: 6m 22s	remaining: 11m 3s
658:	learn 0.7737024388	test 0.7705433318	bestTest 0.7705433318	passed: 0.617 sec	total: 6m 22s	remaining: 11m 3s
659:	learn 0.7735821829	test 0.7704310732	bestTest 0.7705433318	passed: 0.51 sec	total: 6

723:	learn 0.7763240175	test 0.773013022	bestTest 0.773013022	passed: 0.614 sec	total: 7m 1s	remaining: 10m 26s
724:	learn 0.7763721199	test 0.773013022	bestTest 0.773013022	passed: 0.505 sec	total: 7m 1s	remaining: 10m 25s
725:	learn 0.7764683246	test 0.7729007634	bestTest 0.773013022	passed: 0.664 sec	total: 7m 2s	remaining: 10m 25s
726:	learn 0.7764683246	test 0.7731252806	bestTest 0.7731252806	passed: 0.647 sec	total: 7m 3s	remaining: 10m 24s
727:	learn 0.7767569388	test 0.7733497979	bestTest 0.7733497979	passed: 0.575 sec	total: 7m 3s	remaining: 10m 24s
728:	learn 0.7768531435	test 0.7735743152	bestTest 0.7735743152	passed: 0.876 sec	total: 7m 4s	remaining: 10m 23s
729:	learn 0.7769733994	test 0.7736865739	bestTest 0.7736865739	passed: 0.59 sec	total: 7m 5s	remaining: 10m 23s
730:	learn 0.7769493482	test 0.7735743152	bestTest 0.7736865739	passed: 0.5 sec	total: 7m 5s	remaining: 10m 22s
731:	learn 0.7769733994	test 0.7736865739	bestTest 0.7736865739	passed: 0.536 sec	total: 7m 6s	r

795:	learn 0.778921545	test 0.7731252806	bestTest 0.7743601257	passed: 0.601 sec	total: 7m 41s	remaining: 9m 41s
796:	learn 0.7789455962	test 0.7733497979	bestTest 0.7743601257	passed: 0.783 sec	total: 7m 42s	remaining: 9m 41s
797:	learn 0.7790177498	test 0.7732375393	bestTest 0.7743601257	passed: 0.61 sec	total: 7m 42s	remaining: 9m 41s
798:	learn 0.7791380057	test 0.7731252806	bestTest 0.7743601257	passed: 0.558 sec	total: 7m 43s	remaining: 9m 40s
799:	learn 0.779186108	test 0.7734620566	bestTest 0.7743601257	passed: 0.462 sec	total: 7m 43s	remaining: 9m 39s
800:	learn 0.7792582616	test 0.7735743152	bestTest 0.7743601257	passed: 0.626 sec	total: 7m 44s	remaining: 9m 39s
801:	learn 0.7793544663	test 0.7736865739	bestTest 0.7743601257	passed: 0.512 sec	total: 7m 44s	remaining: 9m 38s
802:	learn 0.7793785175	test 0.7735743152	bestTest 0.7743601257	passed: 0.508 sec	total: 7m 45s	remaining: 9m 37s
803:	learn 0.7794987734	test 0.7736865739	bestTest 0.7743601257	passed: 0.587 sec	total: 7m

868:	learn 0.7812064072	test 0.7757072295	bestTest 0.7757072295	passed: 0.653 sec	total: 8m 28s	remaining: 9m 4s
869:	learn 0.7812545096	test 0.7754827122	bestTest 0.7757072295	passed: 0.624 sec	total: 8m 28s	remaining: 9m 4s
870:	learn 0.7812304584	test 0.7754827122	bestTest 0.7757072295	passed: 0.69 sec	total: 8m 29s	remaining: 9m 3s
871:	learn 0.7812304584	test 0.7758194881	bestTest 0.7758194881	passed: 0.662 sec	total: 8m 30s	remaining: 9m 3s
872:	learn 0.7812785608	test 0.7758194881	bestTest 0.7758194881	passed: 0.667 sec	total: 8m 30s	remaining: 9m 2s
873:	learn 0.7813266631	test 0.7758194881	bestTest 0.7758194881	passed: 0.786 sec	total: 8m 31s	remaining: 9m 2s
874:	learn 0.781302612	test 0.7759317467	bestTest 0.7759317467	passed: 0.659 sec	total: 8m 32s	remaining: 9m 1s
875:	learn 0.7814228679	test 0.7759317467	bestTest 0.7759317467	passed: 0.707 sec	total: 8m 33s	remaining: 9m 1s
876:	learn 0.7814709702	test 0.7760440054	bestTest 0.7760440054	passed: 0.591 sec	total: 8m 33s	re

941:	learn 0.7831064505	test 0.7772788505	bestTest 0.777727885	passed: 0.615 sec	total: 9m 11s	remaining: 8m 22s
942:	learn 0.783034297	test 0.7772788505	bestTest 0.777727885	passed: 0.574 sec	total: 9m 12s	remaining: 8m 21s
943:	learn 0.7830583482	test 0.7771665918	bestTest 0.777727885	passed: 0.648 sec	total: 9m 12s	remaining: 8m 21s
944:	learn 0.7831305017	test 0.7772788505	bestTest 0.777727885	passed: 0.534 sec	total: 9m 13s	remaining: 8m 20s
945:	learn 0.783034297	test 0.7772788505	bestTest 0.777727885	passed: 0.611 sec	total: 9m 13s	remaining: 8m 20s
946:	learn 0.7830823993	test 0.7771665918	bestTest 0.777727885	passed: 0.472 sec	total: 9m 14s	remaining: 8m 19s
947:	learn 0.7831305017	test 0.7768298159	bestTest 0.777727885	passed: 0.559 sec	total: 9m 15s	remaining: 8m 18s
948:	learn 0.7831305017	test 0.7769420745	bestTest 0.777727885	passed: 0.582 sec	total: 9m 15s	remaining: 8m 18s
949:	learn 0.7831545529	test 0.7770543332	bestTest 0.777727885	passed: 0.597 sec	total: 9m 16s	rem

In [ ]:
clf.score(xcv, ycv)

In [ ]:
clf.score(xva, yva)